In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
from futile.Utils import write
import numpy as np
import matplotlib.pyplot as plt
import os, sys, yaml
sys.path.insert(0,'../')
import StatPol as SP, Routines as R

basepath = os.getcwd()

Routines basepath : /home/marco/Data/RICERCA/LINEAR RESPONSE/Statical polarizability/Analysis/Calculations and data analysis


## Analysis of the statical polarizability for the nsp dataset

We read the nsp_dataset from file. Note that the file nsp_dataset.yaml contains only the reference results and the information on the allowed studies. So, to update the actual set of results with new computations we have to load
the nsp_results.yaml. In this way the new computations performed will increase the dictionary. 

In [2]:
#nsp_dataset = yaml.load(open('nsp_dataset.yaml'))
nsp_dataset = yaml.load(open('nsp_results.yaml'))
#nsp_dataset

For testing and computation purposes we split the dataset into several subset made of
10 molecules each (molecules are sorted in alphabetical order before the splitting). So we have
8 subsets (the last one with 5 molecules)

In [3]:
molecules = nsp_dataset.keys()
molecules.sort()
print 'number of nsp elements ', len(molecules)
subset_length = 10

subset = {}
for ind in range(int(len(molecules)/subset_length)+1):
    subset[ind] = []

for ind,mol in enumerate(molecules):
    subset[int(ind/subset_length)].append(mol)

number of nsp elements  75


In [9]:
subset[1]

['CH2BH',
 'CH3BH2',
 'CH3Cl',
 'CH3F',
 'CH3NH2',
 'CH3OH',
 'CH3SH',
 'CH4',
 'CO',
 'CO2']

We also define a reduced set of studies (pbe0 is not computed)

In [7]:
reduced_study_set = [('lda','hgh-k'),('pbe','hgh-k'),('pbe','nlcc')] #('pbe0','hgh-k')

Run the nsp_worflow for the nsp_dataset. For each molecule we add the key 'results'. This key contain the results, for all the allowed studies. 

In [10]:
reload(R)

#subset_done = ['N2','CO','Mg2','Mg','H2O','Ar','HCl']
code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 4',skip=True)

for mol in subset[1]:
    data = nsp_dataset[mol]
    data['results'] = {}
    for study in data['study']:
        if study in reduced_study_set:
            data['results'][study] = {}
            data['results'][study] = R.nsp_workflow(term_verb=True,molecule=mol,study=study,code=code)

Routines basepath : /home/marco/Data/RICERCA/LINEAR RESPONSE/Statical polarizability/Analysis/Calculations and data analysis
Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft
Compute alpha for :  CH2BH lda hgh-k
Seek for gs convergence
Perform the run with rmult 3.0
Creating the yaml input file "lda-hgh-k/rmult:3.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:3.0 -s Yes
Perform the run with rmult 4.0
Creating the yaml input file "lda-hgh-k/rmult:4.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:4.0 -s Yes
Convergence for rmult 3.0 failed
Perform the run with rmult 5.0
Creating the yaml input file "lda-hgh-k/rmult:5.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/bina

Perform the run with rmult 5.0
Convergence for rmult 4.0 failed
Perform the run with rmult 6.0
Convergence for rmult 5.0 failed
Perform the run with rmult 7.0
Convergence achieved for rmult 6.0
Compute alpha for :  CH3Cl lda hgh-k
Seek for gs convergence
Perform the run with rmult 3.0
Creating the yaml input file "lda-hgh-k/rmult:3.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:3.0 -s Yes
Perform the run with rmult 4.0
Creating the yaml input file "lda-hgh-k/rmult:4.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:4.0 -s Yes
Convergence for rmult 3.0 failed
Perform the run with rmult 5.0
Creating the yaml input file "lda-hgh-k/rmult:5.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:5.0 -s Yes
Convergence for rmu

Convergence achieved for rmult 5.0
Seek for alpha convergence
Convergence on the field intensity
Perform the run with field_int 0.01
Perform the run with field_int 0.005
Convergence for field_int 0.01 failed
Perform the run with field_int 0.001
Convergence achieved for field_int 0.005
Convergence on rmult
Perform the run with rmult 5.0
Perform the run with rmult 6.0
Convergence for rmult 5.0 failed
Perform the run with rmult 7.0
Convergence achieved for rmult 6.0
Compute alpha for :  CH3F pbe nlcc
Seek for gs convergence
Perform the run with rmult 3.0
Creating the yaml input file "pbe-nlcc/rmult:3.0.yaml"
Run directory pbe-nlcc
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:3.0 -s Yes
Perform the run with rmult 4.0
Creating the yaml input file "pbe-nlcc/rmult:4.0.yaml"
Run directory pbe-nlcc
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:4.0 -s Yes
Convergence for

Convergence achieved for rmult 5.0
Seek for alpha convergence
Convergence on the field intensity
Perform the run with field_int 0.01
Perform the run with field_int 0.005
Convergence achieved for field_int 0.01
Convergence on rmult
Perform the run with rmult 5.0
Perform the run with rmult 6.0
Convergence for rmult 5.0 failed
Perform the run with rmult 7.0
Convergence for rmult 6.0 failed
Perform the run with rmult 8.0
Convergence achieved for rmult 7.0
Compute alpha for :  CH3OH pbe hgh-k
Seek for gs convergence
Perform the run with rmult 3.0
Creating the yaml input file "pbe-hgh-k/rmult:3.0.yaml"
Run directory pbe-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:3.0 -s Yes
Perform the run with rmult 4.0
Creating the yaml input file "pbe-hgh-k/rmult:4.0.yaml"
Run directory pbe-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:4.0 -s Yes
Convergence for rmult

Perform the run with field_int 0.005
Convergence for field_int 0.01 failed
Perform the run with field_int 0.001
Convergence for field_int 0.005 failed
Perform the run with field_int 0.0005
Convergence achieved for field_int 0.001
Convergence on rmult
Perform the run with rmult 5.0
Perform the run with rmult 6.0
Convergence for rmult 5.0 failed
Perform the run with rmult 7.0
Convergence for rmult 6.0 failed
Perform the run with rmult 8.0
Convergence for rmult 7.0 failed
Perform the run with rmult 9.0
Convergence for rmult 8.0 failed
Perform the run with rmult 10.0
Convergence achieved for rmult 9.0
Compute alpha for :  CH4 lda hgh-k
Seek for gs convergence
Perform the run with rmult 3.0
Creating the yaml input file "lda-hgh-k/rmult:3.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:3.0 -s Yes
Perform the run with rmult 4.0
Creating the yaml input file "lda-hgh-k/rmult:4.0.yaml"
Run directory lda-

Convergence achieved for field_int 0.01
Convergence on rmult
Perform the run with rmult 5.0
Perform the run with rmult 6.0
Convergence for rmult 5.0 failed
Perform the run with rmult 7.0
Convergence achieved for rmult 6.0
Compute alpha for :  CO2 lda hgh-k
Seek for gs convergence
Perform the run with rmult 3.0
Creating the yaml input file "lda-hgh-k/rmult:3.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:3.0 -s Yes
Perform the run with rmult 4.0
Creating the yaml input file "lda-hgh-k/rmult:4.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:4.0 -s Yes
Convergence for rmult 3.0 failed
Perform the run with rmult 5.0
Creating the yaml input file "lda-hgh-k/rmult:5.0.yaml"
Run directory lda-hgh-k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:5.0 

In [13]:
#nsp_dataset#['HCl']['results']['pbe','nlcc']

Save the dataset as yaml file

In [14]:
import yaml
with open('nsp_results.yaml', 'w') as outfile:
    yaml.dump(nsp_dataset, outfile, default_flow_style=False)

## Analysis of the statical polarizability for the sp dataset

....TO BE DONE....